### Time gap for bronze table

In [12]:
directory = "//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log"  # Replace with your directory path

import os
import json
import pandas as pd

def get_files(directory):
    files = []
    for file in os.listdir(directory):
        if file.endswith(".json"):
            files.append(file)
    return files

In [18]:
jsonfiles = get_files(directory)
appended_data = []
temp = ''
for file in jsonfiles:
    #merge different and files
    temp = directory + '/' + file
    appended_data.append(temp)


In [19]:
appended_data

['//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000017856.json',
 '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000015954.json',
 '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000002270.json',
 '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000002215.json',
 '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000008886.json',
 '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000012480.json',
 '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000012525

In [31]:
#open the files appended_data[1] and read the data
with open(appended_data[1],'r') as fh:
    data = json.load(fh)
    for i in data:
        print(i)

PermissionError: [Errno 13] Permission denied: '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log/00000000000000015954.json'

In [26]:
import json
import os
from datetime import datetime
import pandas as pd
# Example usage
directory = "/wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log"  # Replace with your directory path
csv_filename = "./time_gap_results.csv"

def process_directory(directory, csv_filename):
  """
  Processes all JSON files in a directory, calculates time gaps, and exports results to a CSV file.

  Args:
      directory: Path to the directory containing JSON files.
      csv_filename: Name of the CSV file to export results.
  """
  columns = ['filename', 'time_gap']
  df=pd.DataFrame(columns=columns)

  for filename in os.listdir(directory):
      data = []
      # convert \ to / in the file path
      if filename.endswith('.json') and filename != "00000000000000000000.json" :
        filepath = os.path.join(directory, filename)
        with open(filepath,'r') as f:
          for line in f:
            #convert \ to / in the file path
            data.append(json.loads(line))
          modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
          stats_json = data[1]["add"]["stats"]
          stats_dict = json.loads(stats_json)
          timestamp_str = stats_dict["minValues"]["timestamp"]
        if timestamp_str:
          timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
          time_gap = modification_time - timestamp.timestamp()
          df = df._append({"filename": filename,
                           "time_gap": time_gap}, ignore_index = True)
        df.to_csv(csv_filename)
        print(f"Average Time gap: {df['time_gap'].mean():.2f}")
      else:
        continue


In [9]:
.
  """
  columns = ['filename', 'time_gap']
  df=pd.DataFrame(columns=columns)
  for filename in os.listdir(directory):
      data = []
      if filename.endswith('.json') and filename != "00000000000000000000.json" :
        filepath = os.path.join(directory, filename)
        with open(filepath,'r') as f:
          for line in f:
            data.append(json.loads(line))

          modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
          stats_json = data[1]["add"]["stats"]
          stats_dict = json.loads(stats_json)
          timestamp_str = stats_dict["minValues"]["timestamp"]
        if timestamp_str:
          timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
          time_gap = modification_time - timestamp.timestamp()
          df = df._append({"filename": filename,
                           "time_gap": time_gap}, ignore_index = True)
        df.to_csv(csv_filename)
        print(f"Average Time gap: {df['time_gap'].mean():.2f}")
      else:
        continue


SyntaxError: unterminated triple-quoted string literal (detected at line 25) (2844718318.py, line 2)

In [27]:
process_directory(directory, csv_filename)

PermissionError: [Errno 13] Permission denied: '//wsl.localhost/docker-desktop-data/data/docker/volumes/basesystem_minio/_data/test/bronze/bitcoin_stg/_delta_log\\00000000000000017856.json'

In [ ]:
import json
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

def process_directory(directory):
    """
    Processes all JSON files in a directory, calculates time gaps, and returns the average time gap.

    Args:
        directory: Path to the directory containing JSON files.

    Returns:
        average_time_gap: The average time gap calculated from all JSON files.
    """
    time_gaps = []
    for filename in os.listdir(directory):
        if filename.endswith('.json') and filename != "00000000000000000000.json":
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as f:
                data = [json.loads(line) for line in f]

            modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
            stats_json = data[1]["add"]["stats"]
            stats_dict = json.loads(stats_json)
            timestamp_str = stats_dict["minValues"]["timestamp"]
            if timestamp_str:
                timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
                time_gap = modification_time - timestamp.timestamp()
                time_gaps.append(time_gap)

    average_time_gap = sum(time_gaps) / len(time_gaps) if time_gaps else 0
    return average_time_gap

def plot_line_chart(directory):
    """
    Plots a line chart for the average delay time metric calculated from the JSON files.

    Args:
        directory: Path to the directory containing JSON files.
    """
    average_time_gaps = []
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            avg_time_gap = process_directory(os.path.join(root, dir))
            average_time_gaps.append(avg_time_gap)

    plt.plot(average_time_gaps)
    plt.xlabel('Folder Index')
    plt.ylabel('Average Time Gap')
    plt.title('Average Delay Time Across Folders')
    plt.show()

# Example usage
directory = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log4/_delta_log"
plot_line_chart(directory)


In [ ]:
import json
import os
from datetime import datetime
import pandas as pd
# Example usage
directory = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log12/_delta_log"  # Replace with your directory path
csv_filename = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log12/time_gap_results.csv"


def process_directory(directory, csv_filename):
  """
  Processes all JSON files in a directory, calculates time gaps, and exports results to a CSV file.

  Args:
      directory: Path to the directory containing JSON files.
      csv_filename: Name of the CSV file to export results.
  """
  columns = ['filename', 'time_gap']
  df=pd.DataFrame(columns=columns)
  for filename in os.listdir(directory):
      data = []
      if filename.endswith('.json') and filename != "00000000000000000000.json" :
        filepath = os.path.join(directory, filename)
        with open(filepath,'r') as f:
          for line in f:
            data.append(json.loads(line))

          modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
          stats_json = data[1]["add"]["stats"]
          stats_dict = json.loads(stats_json)
          timestamp_str = stats_dict["minValues"]["timestamp"]
        if timestamp_str:
          timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
          time_gap = modification_time - timestamp.timestamp()
          df = df._append({"filename": filename,
                           "time_gap": time_gap}, ignore_index = True)
        df.to_csv(csv_filename)
        print(f"Average Time gap: {df['time_gap'].mean():.2f}")
      else:
        continue
import json
import os
from datetime import datetime
import pandas as pd
# Example usage
directory = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log12/_delta_log"  # Replace with your directory path
csv_filename = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log12/time_gap_results.csv"


def process_directory(directory, csv_filename):
  """
  Processes all JSON files in a directory, calculates time gaps, and exports results to a CSV file.

  Args:
      directory: Path to the directory containing JSON files.
      csv_filename: Name of the CSV file to export results.
  """
  columns = ['filename', 'time_gap']
  df=pd.DataFrame(columns=columns)
  for filename in os.listdir(directory):
      data = []
      if filename.endswith('.json') and filename != "00000000000000000000.json" :
        filepath = os.path.join(directory, filename)
        with open(filepath,'r') as f:
          for line in f:
            data.append(json.loads(line))

          modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
          stats_json = data[1]["add"]["stats"]
          stats_dict = json.loads(stats_json)
          timestamp_str = stats_dict["minValues"]["timestamp"]
        if timestamp_str:
          timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
          time_gap = modification_time - timestamp.timestamp()
          df = df._append({"filename": filename,
                           "time_gap": time_gap}, ignore_index = True)
        df.to_csv(csv_filename)
        print(f"Average Time gap: {df['time_gap'].median():.2f}")
      else:
        continue

process_directory(directory,csv_filename)


In [ ]:
import json
import os
from datetime import datetime
import pandas as pd
# Example usage
directory = "D:/Thesis - Dissertation/Performance-Tracking/bit/_delta_log"  # Replace with your directory path
csv_filename = "D:/Thesis - Dissertation/Performance-Tracking/bit/time_gap_results.csv"


def process_directory(directory, csv_filename):
  """
  Processes all JSON files in a directory, calculates time gaps, and exports results to a CSV file.

  Args:
      directory: Path to the directory containing JSON files.
      csv_filename: Name of the CSV file to export results.
  """
  columns = ['filename', 'time_gap']
  df=pd.DataFrame(columns=columns)
  for filename in os.listdir(directory):
      data = []
      if filename.endswith('.json') and filename != "00000000000000000000.json" :
        filepath = os.path.join(directory, filename)
        with open(filepath,'r') as f:
          for line in f:
            data.append(json.loads(line))

          modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
          stats_json = data[1]["add"]["stats"]
          stats_dict = json.loads(stats_json)
          timestamp_str = stats_dict["minValues"]["timestamp"]
        if timestamp_str:
          timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
          time_gap = modification_time - timestamp.timestamp()
          df = df._append({"filename": filename,
                           "time_gap": time_gap}, ignore_index = True)
        df.to_csv(csv_filename)
        print(f"Average Time gap: {df['time_gap'].mean():.2f}")
      else:
        continue

process_directory(directory,csv_filename)


In [ ]:
import json
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

# Example usage
directory = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log11/_delta_log"  # Replace with your directory path
csv_filename = "D:/Thesis - Dissertation/Performance-Tracking/new/bronze/bitcoin_delta_log11/time_gap_results.csv"


def process_directory(directory, csv_filename):
    """
    Processes all JSON files in a directory, calculates time gaps, and exports results to a CSV file.

    Args:
        directory: Path to the directory containing JSON files.
        csv_filename: Name of the CSV file to export results.
    """
    columns = ['filename', 'time_gap']
    df = pd.DataFrame(columns=columns)
    for filename in os.listdir(directory):
        data = []
        if filename.endswith('.json') and filename != "00000000000000000000.json":
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as f:
                for line in f:
                    data.append(json.loads(line))

                modification_time = data[1]['add']['modificationTime'] / 1000  # Convert to seconds
                stats_json = data[1]["add"]["stats"]
                stats_dict = json.loads(stats_json)
                timestamp_str = stats_dict["minValues"]["timestamp"]
            if timestamp_str:
                timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%fZ")
                time_gap = modification_time - timestamp.timestamp()
                df = df._append({"filename": filename,
                                "time_gap": time_gap}, ignore_index=True)
    df.to_csv(csv_filename)
    print(f"Average Time gap: {df['time_gap'].mean():.2f}")

    # Plotting
    plt.figure(figsize=(25, 10))
    plt.plot(df['time_gap'], marker='o', linestyle='-')
    plt.title('Average Time Gap Over Files')
    plt.xlabel('File Index')
    plt.ylabel('Average Time Gap')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


process_directory(directory, csv_filename)


OSError: [Errno 22] Invalid argument